In [ ]:
# Bitcoin Mainnet - Propagar "XD" via mensaje ADDR (propagación viral)
# Los nodos reenvían addr a sus peers automáticamente

import socket
import struct
import hashlib
import time
import random

MAINNET_MAGIC = 0xD9B4BEF9
MAINNET_SEEDS = [
    "seed.bitcoin.sipa.be",
    "dnsseed.bluematt.me",
    "seed.bitcoinstats.com",
    "seed.bitcoin.jonasschnelli.ch"
]
PORT = 8333
PROTOCOL_VERSION = 70016

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

def create_message(command, payload=b''):
    magic = struct.pack('<I', MAINNET_MAGIC)
    cmd = command.encode('ascii').ljust(12, b'\x00')
    length = struct.pack('<I', len(payload))
    chk = double_sha256(payload)[:4]
    return magic + cmd + length + chk + payload

def create_version_payload():
    version = struct.pack('<i', PROTOCOL_VERSION)
    services = struct.pack('<Q', 1)  # NODE_NETWORK - parecemos nodo real
    timestamp = struct.pack('<q', int(time.time()))
    addr_recv = struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', PORT)
    addr_from = struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', PORT)
    nonce = struct.pack('<Q', random.randint(0, 2**64-1))
    user_agent = b'\x10/XD-Propagator:1/'
    start_height = struct.pack('<i', 800000)
    relay = struct.pack('<?', True)
    return version + services + timestamp + addr_recv + addr_from + nonce + user_agent + start_height + relay

def create_xd_addr_payload():
    """
    Crear mensaje ADDR con IPs que codifican "XD"
    Los nodos propagan esto automáticamente a sus peers!

    IP: 88.68.x.x = "XD" en ASCII (X=88, D=68)
    """
    entries = []

    # Crear múltiples "direcciones" con el patrón XD
    xd_ips = [
        (88, 68, 88, 68),    # XD.XD
        (88, 68, 0, 1),      # XD.0.1
        (88, 68, 0, 2),      # XD.0.2
        (88, 68, 1, 1),      # XD.1.1
        (88, 68, 13, 37),    # XD.13.37 (leet)
    ]

    for ip_parts in xd_ips:
        entry = b''
        entry += struct.pack('<I', int(time.time()))  # timestamp reciente = se propaga
        entry += struct.pack('<Q', 1)  # services = NODE_NETWORK
        entry += b'\x00' * 10 + b'\xff\xff'  # IPv4-mapped IPv6
        entry += bytes(ip_parts)  # IP con "XD" codificado
        entry += struct.pack('>H', 8333)  # puerto estándar
        entries.append(entry)

    # Varint count + entries
    count = len(entries)
    payload = bytes([count]) + b''.join(entries)
    return payload

def parse_message(data):
    if len(data) < 24:
        return None, None, data
    magic = struct.unpack('<I', data[:4])[0]
    if magic != MAINNET_MAGIC:
        return None, None, data
    command = data[4:16].rstrip(b'\x00').decode('ascii', errors='ignore')
    payload_len = struct.unpack('<I', data[16:20])[0]
    if len(data) < 24 + payload_len:
        return None, None, data
    payload = data[24:24+payload_len]
    remaining = data[24+payload_len:]
    return command, payload, remaining

def propagate_via_addr(ip, port):
    """Conectar a un nodo y enviar ADDR con XD codificado"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(15)

    try:
        print(f"\n🔌 Conectando a {ip}:{port}...")
        sock.connect((ip, port))
        print(f"✅ Conectado!")

        # Handshake
        sock.send(create_message('version', create_version_payload()))
        print("📤 VERSION enviado")

        buffer = b''
        handshake_done = False
        start = time.time()

        while time.time() - start < 10:
            try:
                data = sock.recv(4096)
                if not data:
                    break
                buffer += data

                while True:
                    cmd, payload, buffer = parse_message(buffer)
                    if cmd is None:
                        break

                    print(f"📥 {cmd}")

                    if cmd == 'version':
                        sock.send(create_message('verack'))
                        print("📤 VERACK")

                    elif cmd == 'verack':
                        handshake_done = True
                        print("🤝 ¡HANDSHAKE COMPLETO!")

                    elif cmd == 'ping':
                        sock.send(create_message('pong', payload[:8]))

            except socket.timeout:
                if handshake_done:
                    break

        if not handshake_done:
            print("❌ Handshake falló")
            sock.close()
            return False

        # ¡ENVIAR EL ADDR CON XD!
        print("\n" + "="*50)
        print("🚀 ENVIANDO ADDR CON 'XD' CODIFICADO")
        print("="*50)
        print("   IPs que se propagarán:")
        print("   • 88.68.88.68 (XD.XD)")
        print("   • 88.68.0.1   (XD.0.1)")
        print("   • 88.68.0.2   (XD.0.2)")
        print("   • 88.68.1.1   (XD.1.1)")
        print("   • 88.68.13.37 (XD.leet)")
        print()

        addr_payload = create_xd_addr_payload()
        addr_msg = create_message('addr', addr_payload)
        sock.send(addr_msg)
        print("📤 ¡ADDR enviado!")
        print()
        print("🔄 Este nodo DEBERÍA propagar estas IPs a sus peers")
        print("   Y esos peers a los suyos... efecto viral!")

        # Esperar respuestas
        print("\n⏳ Esperando actividad...")
        sock.settimeout(3)
        for _ in range(5):
            try:
                data = sock.recv(4096)
                if data:
                    cmd, _, _ = parse_message(data)
                    if cmd:
                        print(f"📥 Respuesta: {cmd}")
            except:
                pass

        sock.close()
        return True

    except Exception as e:
        print(f"❌ Error: {e}")
        sock.close()
        return False

def main():
    print("="*60)
    print("🌐 BITCOIN MAINNET - PROPAGADOR VIRAL 'XD' via ADDR")
    print("="*60)
    print()
    print("📋 CÓMO FUNCIONA:")
    print("   1. Conectamos a nodos reales de Bitcoin")
    print("   2. Enviamos mensaje 'addr' con IPs especiales")
    print("   3. IP 88.68.x.x = 'XD' en ASCII (X=88, D=68)")
    print("   4. Los nodos propagan 'addr' a TODOS sus peers")
    print("   5. Efecto viral: cada nodo lo reenvía")
    print()

    # Obtener peers de DNS seeds
    print("📡 Resolviendo DNS seeds...")
    peers = []
    for seed in MAINNET_SEEDS:
        try:
            ips = socket.gethostbyname_ex(seed)[2]
            peers.extend([(ip, PORT) for ip in ips[:5]])
            print(f"   ✅ {seed}: {len(ips)} IPs")
            if len(peers) >= 10:
                break
        except Exception as e:
            print(f"   ❌ {seed}: {e}")

    random.shuffle(peers)

    # Conectar a múltiples nodos para máxima propagación
    print(f"\n🎯 Propagando a múltiples nodos para efecto viral...")

    successful = 0
    for ip, port in peers[:5]:
        if propagate_via_addr(ip, port):
            successful += 1
            print(f"   ✅ Propagado via {ip}")
        time.sleep(1)

    # Resumen
    print("\n" + "="*60)
    print("📊 RESUMEN DE PROPAGACIÓN")
    print("="*60)
    print(f"   ✅ Nodos infectados directamente: {successful}")
    print(f"   🔄 Propagación esperada: VIRAL")
    print()
    print("   📝 Lo que enviamos:")
    print("      Mensaje 'addr' con IPs: 88.68.x.x")
    print("      88 = 'X' en ASCII")
    print("      68 = 'D' en ASCII")
    print()
    print("   🌐 Los nodos propagarán estas 'direcciones' a sus peers")
    print("      pensando que son nodos Bitcoin reales.")
    print()
    print("   ⚠️ NOTA: Eventualmente los nodos intentarán conectar")
    print("      a 88.68.x.x, fallarán, y las eliminarán.")
    print("      Pero por un tiempo, 'XD' viajará por la red!")
    print("="*60)

main()